In [1]:
import os
import gc
import sys
import warnings
from pathlib import Path

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose

In [2]:
df_names = ['playerBoxScores', 'teamBoxScores', 'playerTwitterFollowers', 'teamTwitterFollowers', 'awards', 'events',
           'games', 'standings', 'rosters', 'transactions']
path = "data/train_vs/"

train_dict = {}
for index in range(len(df_names)):
    train_dict[df_names[index]] = pd.read_pickle(os.path.join(path,df_names[index]) + "_train.pkl")

In [3]:
df_names = ['seasons', 'teams', 'players', 'awards']
path = "data/"

my_dict = {}
for index in range(len(df_names)):
    my_dict[df_names[index]] = pd.read_csv(os.path.join(path,df_names[index]) + ".csv")

In [4]:
target_v = pd.read_pickle('data/target_v/nextDayPlayerEngagement_train.pkl')

In [5]:
target_dict = {id: target_v[target_v['playerId']==id].drop(
    ['index'], axis = 1) for id in target_v['playerId'].unique()}    

In [6]:
target_dict[112526].head(3)

,engagementMetricsDate,playerId,target1,target2,target3,target4,date
969,2018-01-02,112526,0.055277,5.496109,0.025839,16.176471,20180101
3556,2018-01-03,112526,0.060625,3.252914,0.030486,8.541353,20180102
5737,2018-01-04,112526,0.029341,1.648352,0.032613,10.490112,20180103


## Dictionary for playerBoxScores

In [7]:
perform_stats = pd.DataFrame()
perform_stats = train_dict['playerBoxScores'][[
    'playerId', 'gamePk', 'gameDate', 'gameTimeUTC', 'teamId', 'date']]
perform_stats.head(3)

,playerId,gamePk,gameDate,gameTimeUTC,teamId,date
0,605131,529418,2018-03-29,2018-03-29T23:08:00Z,119,20180329
1,605480,529406,2018-03-29,2018-03-29T20:00:00Z,139,20180329
2,546318,529416,2018-03-29,2018-03-29T20:10:00Z,143,20180329


In [8]:
performance = train_dict['playerBoxScores']

In [9]:
perform_stats['FPCT'] = (performance['putOuts']+performance['assists']) / performance['chances']

# Not sure
perform_stats['Out'] = performance['putOuts'] + performance['groundOuts'] + performance['strikeOuts'] + performance['flyOuts']

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [10]:
perform_stats['AB'] = performance['atBats']

# Not sure
perform_stats['B2'] = performance['doubles']
perform_stats['B3'] = performance['triples']

perform_stats['SB'] = performance['stolenBases']
perform_stats['CS'] = performance['caughtStealing']
perform_stats['H'] = performance['hits']
perform_stats['HR'] = performance['homeRuns']
perform_stats['IBB'] = performance['intentionalWalks']
perform_stats['HBP'] = performance['hitByPitch']
perform_stats['LOB'] = performance['leftOnBase']
perform_stats['PA'] = performance['plateAppearances']
perform_stats['TB'] = performance['totalBases']
perform_stats['GIDP'] = performance['groundIntoDoublePlay']
perform_stats['RBI'] = performance['rbi']
perform_stats['BB'] = performance['baseOnBalls']
perform_stats['SO'] = performance['strikeOuts']

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

In [11]:
perform_stats['SV_percnt'] = performance['saves']/performance['saveOpportunities']
perform_stats['HLD'] = performance['holds']
perform_stats['BS'] = performance['blownSaves']
perform_stats['IR'] = performance['inheritedRunners']
perform_stats['IP'] = performance['inningsPitched']
perform_stats['ERA'] = (9*performance['earnedRuns'])/perform_stats['IP']
perform_stats['WHIP'] = (perform_stats['BB'] + perform_stats['H'])/perform_stats['IP']

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

In [202]:
perform_stats['IP'].unique()

array([nan, 0. , 0.1, 0.2, 1. , 1.1, 1.2, 2. , 3.1, 4. , 4.1, 4.2, 5. ,
       5.1, 5.2, 6. , 7. , 8. , 2.1, 2.2, 3. , 3.2, 6.1, 7.2, 9. , 6.2,
       7.1, 8.1, 8.2])

In [12]:
perform_stats['ISO'] = (perform_stats['B2']+2*perform_stats['B3']+3*perform_stats['HR'])/perform_stats['AB']

perform_stats['RC'] = perform_stats['TB']*(perform_stats['H']+perform_stats['BB'])/(
    perform_stats['AB']+perform_stats['BB'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [13]:
playerBS_dict = {id: perform_stats[perform_stats[
    'playerId']==id] for id in target_v['playerId'].unique()}    

In [14]:
merge_1  = {}
for key, df in target_dict.items():
    df = target_dict[key].merge(playerBS_dict[key], on = ['date', 'playerId'], how = 'left')
    merge_1[key] = df

In [15]:
train_dict.keys()

dict_keys(['playerBoxScores', 'teamBoxScores', 'playerTwitterFollowers', 'teamTwitterFollowers', 'awards', 'events', 'games', 'standings', 'rosters', 'transactions'])

In [16]:
# train_dict['teamBoxScores'].columns

In [17]:
select_awards = train_dict['awards'][['playerId', 'date', 'awardId']]

In [18]:
awards_dict = {id: select_awards[select_awards[
    'playerId']==id] for id in target_v['playerId'].unique()}    

In [19]:
merge_2  = {}
for key, df in target_dict.items():
    df = merge_1[key].merge(awards_dict[key], on = ['date', 'playerId'], how = 'left')
    merge_2[key] = df

In [20]:
select_stand = train_dict['standings'][['gameDate', 'divisionId', 'teamId', 'divisionRank', 'leagueRank', 'wildCardRank', 
                                         'pct', 'homeWins', 'xWinLossPct', 'date']]

In [21]:
# train_dict['standings'].head()
merge_3  = {}
for key, df in target_dict.items():
    df = merge_2[key].merge(select_stand, on = ['date', 'teamId', 'gameDate'], how = 'left')
    merge_3[key] = df

In [22]:
train_dict['rosters'].head(3)

,playerId,gameDate,teamId,statusCode,status,index,date
0,400121,2018-01-01,116,A,Active,0,20180101
1,408045,2018-01-01,142,A,Active,0,20180101
2,425492,2018-01-01,120,A,Active,0,20180101


In [23]:
select_rosters = train_dict['rosters'].drop(['status', 'index'], axis = 1)

In [24]:
rosters_dict = {id: select_rosters[select_rosters['playerId']==id] for id in target_v['playerId'].unique()}

In [25]:
merge_4  = {}
for key, df in target_dict.items():
    df = merge_3[key].merge(rosters_dict[key], on = ['date', 'playerId', 'teamId', 'gameDate'], how = 'left')
    merge_4[key] = df

In [26]:
select_games = train_dict['games'][['gamePk', 'gameType', 'gameTimeUTC', 'codedGameState', 'gameNumber',
                                   'homeId', 'homeWinPct', 'homeScore',]]

In [27]:
merge_5  = {}
for key, df in target_dict.items():
    df = merge_4[key].merge(select_games, on = ['gameTimeUTC', 'gamePk'], how = 'left')
    merge_5[key] = df

In [61]:
# train_dict['events'].columns

In [44]:
select_events = train_dict['events'][['gamePk', 'gameDate', 'gameTimeUTC','gameType', 'eventId', 
                                      'inning', 'halfInning', 'homeScore', 'pitcherTeamId', 
                                     'hitterTeamId', 'pitchType', 'pitchType', 'date']]

In [ ]:
# merge_6  = {}
# for key, df in target_dict.items():
#     df = merge_5[key].merge(select_events, on = ['gamePk', 'gameDate', 'gameTimeUTC','gameType'], how = 'left')
#     merge_6[key] = df

## Construct whole df

In [84]:
df = pd.concat(df_list ,axis=0)

In [89]:
df = df.drop(['gamePk', 'gameDate', 'gameTimeUTC'], axis = 1)

In [124]:
object_encode = df.select_dtypes(include=['object']).drop(['engagementMetricsDate', 'pct'], axis = 1)

In [ ]:
df['pct'] = df['pct'].astype('float')

In [126]:
def label_encode(df, column):
    func = {c: i for i, c in enumerate(df[column].unique()) if not pd.isna(c)}
    df[column] = df[column].map(func)

In [127]:
for col in object_encode.columns:
    label_encode(object_encode, col)

In [130]:
df[object_encode.columns] = object_encode

In [159]:
label_encode(df, 'date')

In [142]:
df = df.drop(['teamId'], axis = 1)
df = df.fillna(-99)
df = df[df.columns.difference(['engagementMetricsDate'])].astype('float32')

## RandomForest Model

In [87]:
from sklearn.ensemble import RandomForestRegressor

In [180]:
X_pre = df[df.columns.difference(['engagementMetricsDate', 'target1', 'target2', 'target3', 'target4'])]
y = df['target1']

In [181]:
X_dict = {id: X_pre[X_pre['playerId']==id] for id in target_v['playerId'].unique()} 

In [182]:
regr = RandomForestRegressor(max_depth=8, max_features = 18, min_samples_split = 6, n_estimators = 15, random_state=0)

In [189]:
X_dict.keys()

dict_keys([628317, 547989, 519317, 607625, 592547, 641553, 606959, 435263, 502179, 493200, 605490, 517369, 543939, 460077, 543768, 476454, 542979, 461865, 641857, 664208, 572998, 519203, 453568, 608701, 593700, 425772, 592200, 571760, 592811, 542583, 593144, 594795, 544928, 641531, 641490, 642180, 641432, 570799, 622097, 477132, 446334, 452104, 622761, 501571, 608349, 459429, 571718, 642162, 614177, 457918, 605195, 594992, 448281, 434538, 435079, 516782, 592879, 502671, 548389, 488681, 623364, 592340, 553878, 461314, 592387, 627894, 643338, 642571, 425877, 605508, 501659, 460269, 571946, 593423, 465657, 592169, 547004, 501817, 605538, 453943, 570240, 545338, 500743, 605253, 641645, 543807, 624577, 622491, 607455, 488671, 581527, 624133, 660623, 519455, 455119, 645261, 607680, 605164, 543228, 622713, 474832, 453172, 571666, 606299, 605288, 592454, 433587, 592863, 444432, 460026, 596001, 518516, 592791, 596115, 571918, 501985, 596057, 504379, 570663, 553869, 543243, 657205, 570482, 49324

In [192]:
tx = X_dict[628317].drop(['playerId'], axis = 1)

In [194]:
tx = round(tx,2)

In [198]:
df.max()

AB                     8.0
B2                     4.0
B3                     2.0
BB                     5.0
BS                     1.0
CS                     2.0
ERA                    inf
FPCT                   1.0
GIDP                   3.0
H                      6.0
HBP                    3.0
HLD                    1.0
HR                     3.0
IBB                    3.0
IP                     9.0
IR                     3.0
ISO                    3.0
LOB                   11.0
Out                   29.0
PA                     9.0
RBI                   10.0
RC                    16.0
SB                     4.0
SO                     5.0
SV_percnt              1.0
TB                    16.0
WHIP                   inf
awardId              290.0
codedGameState         1.0
date                 658.0
divisionId           205.0
divisionRank           5.0
gameNumber             2.0
gameType               6.0
homeId               160.0
homeScore             29.0
homeWinPct             1.0
h

In [195]:
regr.fit(tx,y)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

### Add twitter columns 

In [ ]:
for i in train_dict['teamTwitterFollowers']['teamId'].unique():
    